In [125]:
import os
import torch
import numpy as np
from pathlib import Path

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader,Dataset
from PIL import Image

In [3]:
#### Config Values ###

IMG_SHAPE = (224,224,3)
BATCH_SIZE = 32
EPOCHS = 40
SHUFFLE = True
SEED = 42
CLASSES =['cats'] # for OC-CNN
CLASSES_T =['unknown','cats','dogs'] # for BoN-CNN
CLASSES_B =['wild','cats','dogs'] # for Bon test
TRAIN_PATH = r"C:\Users\asiclab06\Datasets\afhq\train"
TEST_PATH = r"C:\Users\asiclab06\Datasets\afhq\val"

In [141]:
## Define Dataset
from torchvision.transforms import transforms

class ImageDataset(Dataset):
    def __init__(self,root,classes,target_size:tuple,shuffle=False,**kwargs):
        super().__init__()
        self.root=  root
        self.classes = classes
        self.target_size = target_size
        self.shuffle = shuffle        

        self._load_data()
        self._set_index_array()     
        self._set_shuffle()
    
        # kwargs
        self.samples = kwargs.pop('samples',None)
        if self.samples is not None:
            self._sampling_data()

    def _sampling_data(self):
        s = np.random.choice(self.__len__()-self.samples,1).item() if self.shuffle else 0
        e = self.samples
        self.data = self.data[s:e+s]
        self.label = self.label[s:e+s]

    def _load_data(self):
        self.data = []
        self.label = []

        for i,cla in enumerate(self.classes):
            sub_dir = os.path.join(self.root,cla)
            sub_files = os.listdir(sub_dir)
            self.data.extend([os.path.join(sub_dir,sub_file) for sub_file in sub_files])
            l=len(sub_files)
            self.label.extend([i for _ in range(l)])
            print(f'Found {l} images in "{cla}" directory.')

        self.data = np.array(self.data) 
        self.label = np.array(self.label)
        
    def _set_index_array(self):
        self.index_array = np.arange(self.__len__())
        if self.shuffle:
            self.index_array = np.random.permutation(self.__len__())

    def _set_shuffle(self):
        self.data = self.data[self.index_array]
        self.label = self.label[self.index_array]

    def __getitem__(self, index):
        """
            Return : PIL image, sparse label
        """
        x = Image.open(self.data[index]).convert('RGB')
        x = x.resize(self.target_size,Image.Resampling.BILINEAR)
        
        return x, self.label[index]

    def __len__(self):
        return len(self.data)

class FaceClfDataset(ImageDataset):
    def __init__(self, root, classes, target_size: tuple, shuffle=False, transform=None, class_mode=None, **kwargs):
        super().__init__(root, classes, target_size, shuffle, **kwargs)
        # transform
        if transform is not None:
            self.transform = transform
        else:
            self.transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
            ])

        self.allow_class_modes =  ["categorical","sparse",None]

        if class_mode is not None:
            self.class_mode = class_mode
        else:
            self.class_mode = self.allow_class_modes[0]

    def _class_mode(self,y):
        if self.class_mode == self.allow_class_modes[0]:
            return self.to_categorical_class(y)
        elif self.class_mode == self.allow_class_modes[1]:
            return y

    def to_categorical_class(self,y):
        temp = np.zeros()
        # todo
        
    def __getitem__(self, index):
        x,y = super().__getitem__()


        return 

    def __len__(self):
        return super().__len__()    

In [101]:
a = np.array([1,2,3,4,5,6,7,8,9])
b = np.array([5,6,8,7,9,4,3,2,1]) -1


# c = np.random.uniform(0,len(a),len(a))
# c
a[8]


"{:.4f}".format(0.12323231312)

'0.1232'

In [102]:
np.random.choice(1,1)

array([0])

In [138]:

test = ImageDataset(TRAIN_PATH,['cats','dogs'],(224,224),shuffle=False,samples=600)
len(test.label)

Found 300 images in "cats" directory.
Found 300 images in "dogs" directory.


600

In [139]:
loader_test = DataLoader(test,1,False)

In [140]:
iter(loader_test).__next__()

[tensor([[[[203, 164,  87],
           [205, 166,  89],
           [209, 170,  93],
           ...,
           [245, 204, 120],
           [241, 202, 122],
           [239, 200, 121]],
 
          [[203, 164,  87],
           [205, 166,  89],
           [209, 170,  93],
           ...,
           [245, 204, 121],
           [242, 203, 122],
           [240, 201, 122]],
 
          [[203, 164,  87],
           [205, 166,  89],
           [209, 170,  93],
           ...,
           [245, 204, 122],
           [243, 204, 124],
           [241, 202, 123]],
 
          ...,
 
          [[154, 123,  56],
           [155, 124,  57],
           [156, 125,  58],
           ...,
           [  3,   3,   1],
           [  3,   3,   1],
           [  3,   3,   1]],
 
          [[153, 122,  55],
           [153, 122,  55],
           [154, 123,  56],
           ...,
           [  2,   2,   0],
           [  2,   2,   0],
           [  2,   2,   0]],
 
          [[151, 120,  53],
           [152, 121